In [1]:
import requests
import pandas as pd
import json
from google.cloud import bigquery, storage
from google.oauth2.service_account import Credentials

In [2]:
api_key="bfc39b36158f668bf1412ec68685cd67"
base_url="https://api.openweathermap.org/data/2.5/"

In [3]:
cities = pd.read_csv('cities in mindanao.csv')
cities

,Name,Latitude,Longitude
0,Cagayan de Oro,8.475642,124.642153
1,Davao,7.064831,125.608062
2,Zamboanga,6.904691,122.076462
3,Butuan,8.948332,125.536944
4,General Santos,6.112331,125.171714
5,Cotabato,7.223763,124.246706
6,Mati,6.951406,126.216727
7,Valencia,7.902884,125.089803


In [28]:
def fetch(latitude: float, longitude: float) -> json:
    forecast = requests.get(f"{base_url}/weather?lat={latitude}&lon={longitude}&appid={api_key}")
    if forecast.status_code==200:
        print(forecast.json())
        return forecast.json()

### Forecast

In [3]:
test = requests.get("https://api.openweathermap.org/data/2.5/forecast?lat=7.0648306&lon=125.6080623&appid=bfc39b36158f668bf1412ec68685cd67&units=metric")

In [4]:
test.status_code

200

In [5]:
test.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1682456400,
   'main': {'temp': 25.75,
    'feels_like': 26.6,
    'temp_min': 25.16,
    'temp_max': 25.75,
    'pressure': 1008,
    'sea_level': 1008,
    'grnd_level': 1008,
    'humidity': 85,
    'temp_kf': 0.59},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'clouds': {'all': 41},
   'wind': {'speed': 2.31, 'deg': 334, 'gust': 2.59},
   'visibility': 10000,
   'pop': 0.25,
   'rain': {'3h': 0.13},
   'sys': {'pod': 'n'},
   'dt_txt': '2023-04-25 21:00:00'},
  {'dt': 1682467200,
   'main': {'temp': 26.88,
    'feels_like': 29.48,
    'temp_min': 26.88,
    'temp_max': 27.29,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 1009,
    'humidity': 80,
    'temp_kf': -0.41},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': {'all': 63},
   'wind': {'speed': 2.81, 'deg': 12, 'gust':

In [9]:
test.json().keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [6]:
test.json()['city']

{'id': 1715348,
 'name': 'Davao City',
 'coord': {'lat': 7.0648, 'lon': 125.6081},
 'country': 'PH',
 'population': 1212504,
 'timezone': 28800,
 'sunrise': 1682457910,
 'sunset': 1682502329}

In [7]:
test_df=pd.DataFrame.from_dict(test.json()['list'])
test_df.head()

,dt,main,weather,clouds,wind,visibility,pop,rain,sys,dt_txt
0,1682456400,"{'temp': 25.75, 'feels_like': 26.6, 'temp_min'...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 41},"{'speed': 2.31, 'deg': 334, 'gust': 2.59}",10000,0.25,{'3h': 0.13},{'pod': 'n'},2023-04-25 21:00:00
1,1682467200,"{'temp': 26.88, 'feels_like': 29.48, 'temp_min...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 63},"{'speed': 2.81, 'deg': 12, 'gust': 3.4}",10000,0.21,NaN,{'pod': 'd'},2023-04-26 00:00:00
2,1682478000,"{'temp': 29.98, 'feels_like': 33.84, 'temp_min...","[{'id': 802, 'main': 'Clouds', 'description': ...",{'all': 48},"{'speed': 2.1, 'deg': 76, 'gust': 2.22}",10000,0.00,NaN,{'pod': 'd'},2023-04-26 03:00:00
3,1682488800,"{'temp': 30.17, 'feels_like': 34.72, 'temp_min...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 51},"{'speed': 3.37, 'deg': 145, 'gust': 2.93}",10000,0.23,NaN,{'pod': 'd'},2023-04-26 06:00:00
4,1682499600,"{'temp': 28.85, 'feels_like': 33.52, 'temp_min...","[{'id': 500, 'main': 'Rain', 'description': 'l...",{'all': 89},"{'speed': 3.3, 'deg': 138, 'gust': 3.98}",10000,0.41,{'3h': 0.12},{'pod': 'd'},2023-04-26 09:00:00


In [8]:
test_df=pd.json_normalize(
    test.json()['list'],
    'weather',
    ['dt',
     'dt_txt',
     'visibility',
     ['main', 'temp'],
     ['main', 'feels_like'],
     ['main', 'temp_min'],
     ['main', 'temp_max'],
     ['main', 'pressure'],
     ['main', 'sea_level'],
     ['main', 'grnd_level'],
     ['main', 'humidity'],
     ['main', 'temp_kf'],
     'pop',
     ['clouds', 'all'],
     ['rain', '3h'],
     ['sys', 'pod'],
     ['wind', 'speed'],
     ['wind', 'deg'],
     ['wind', 'gust']
    ],
    errors='ignore',
    record_prefix='main_'
)
test_df['city'] = test.json()['city']['name']
test_df['latitude'] = test.json()['city']['coord']['lat']
test_df['longitude'] = test.json()['city']['coord']['lon']
test_df['country'] = test.json()['city']['country']
test_df['timezone'] = test.json()['city']['timezone']
test_df['sunrise'] = test.json()['city']['sunrise']
test_df['sunrise'] = test.json()['city']['sunrise']
test_df

,main_id,main_main,main_description,main_icon,dt,dt_txt,visibility,main.temp,main.feels_like,main.temp_min,...,sys.pod,wind.speed,wind.deg,wind.gust,city,latitude,longitude,country,timezone,sunrise
0,500,Rain,light rain,10n,1682456400,2023-04-25 21:00:00,10000,25.75,26.6,25.16,...,n,2.31,334,2.59,Davao City,7.0648,125.6081,PH,28800,1682457910
1,803,Clouds,broken clouds,04d,1682467200,2023-04-26 00:00:00,10000,26.88,29.48,26.88,...,d,2.81,12,3.4,Davao City,7.0648,125.6081,PH,28800,1682457910
2,802,Clouds,scattered clouds,03d,1682478000,2023-04-26 03:00:00,10000,29.98,33.84,29.98,...,d,2.1,76,2.22,Davao City,7.0648,125.6081,PH,28800,1682457910
3,803,Clouds,broken clouds,04d,1682488800,2023-04-26 06:00:00,10000,30.17,34.72,30.17,...,d,3.37,145,2.93,Davao City,7.0648,125.6081,PH,28800,1682457910
4,500,Rain,light rain,10d,1682499600,2023-04-26 09:00:00,10000,28.85,33.52,28.85,...,d,3.3,138,3.98,Davao City,7.0648,125.6081,PH,28800,1682457910
5,500,Rain,light rain,10n,1682510400,2023-04-26 12:00:00,10000,27.42,30.7,27.42,...,n,1.43,161,2.52,Davao City,7.0648,125.6081,PH,28800,1682457910
6,500,Rain,light rain,10n,1682521200,2023-04-26 15:00:00,10000,26.4,26.4,26.4,...,n,1.41,355,2.11,Davao City,7.0648,125.6081,PH,28800,1682457910
7,500,Rain,light rain,10n,1682532000,2023-04-26 18:00:00,10000,25.35,26.21,25.35,...,n,2.84,338,3.4,Davao City,7.0648,125.6081,PH,28800,1682457910
8,804,Clouds,overcast clouds,04n,1682542800,2023-04-26 21:00:00,10000,24.76,25.54,24.76,...,n,3.3,339,4.42,Davao City,7.0648,125.6081,PH,28800,1682457910
9,804,Clouds,overcast clouds,04d,1682553600,2023-04-27 00:00:00,10000,27.26,30.11,27.26,...,d,3.53,3,5.01,Davao City,7.0648,125.6081,PH,28800,1682457910


In [9]:
test_df.columns = test_df.columns.str.replace(".", "_", regex=False)
test_df.columns

Index(['main_id', 'main_main', 'main_description', 'main_icon', 'dt', 'dt_txt',
       'visibility', 'main_temp', 'main_feels_like', 'main_temp_min',
       'main_temp_max', 'main_pressure', 'main_sea_level', 'main_grnd_level',
       'main_humidity', 'main_temp_kf', 'pop', 'clouds_all', 'rain_3h',
       'sys_pod', 'wind_speed', 'wind_deg', 'wind_gust', 'city', 'latitude',
       'longitude', 'country', 'timezone', 'sunrise'],
      dtype='object')

In [10]:
column_names = {
    'sys_pod': 'part_of_day',
    'dt': 'date_unix',
    'dt_txt': 'date_text',
    'pop': 'precipitation_probability',
    'wind_deg': 'wind_direction',
    'main_main': 'weather',
    'main_sea_level': 'sea_level_pressure',
    'main_grnd_level': 'ground_level_pressure'
}
test_df.rename(columns=column_names, inplace=True)

In [13]:
pd.to_datetime(test_df['sunrise'],unit='s')

0    2023-04-25 21:25:10
1    2023-04-25 21:25:10
2    2023-04-25 21:25:10
3    2023-04-25 21:25:10
4    2023-04-25 21:25:10
5    2023-04-25 21:25:10
6    2023-04-25 21:25:10
7    2023-04-25 21:25:10
8    2023-04-25 21:25:10
9    2023-04-25 21:25:10
10   2023-04-25 21:25:10
11   2023-04-25 21:25:10
12   2023-04-25 21:25:10
13   2023-04-25 21:25:10
14   2023-04-25 21:25:10
15   2023-04-25 21:25:10
16   2023-04-25 21:25:10
17   2023-04-25 21:25:10
18   2023-04-25 21:25:10
19   2023-04-25 21:25:10
20   2023-04-25 21:25:10
21   2023-04-25 21:25:10
22   2023-04-25 21:25:10
23   2023-04-25 21:25:10
24   2023-04-25 21:25:10
25   2023-04-25 21:25:10
26   2023-04-25 21:25:10
27   2023-04-25 21:25:10
28   2023-04-25 21:25:10
29   2023-04-25 21:25:10
30   2023-04-25 21:25:10
31   2023-04-25 21:25:10
32   2023-04-25 21:25:10
33   2023-04-25 21:25:10
34   2023-04-25 21:25:10
35   2023-04-25 21:25:10
36   2023-04-25 21:25:10
37   2023-04-25 21:25:10
38   2023-04-25 21:25:10
39   2023-04-25 21:25:10


In [53]:
filename='test.json'
test_df.to_json(
    filename,
    orient="records",
    lines=True
)

In [44]:
project_id = 'level-agent-375808'
bucket_name = "sample-weather"
bq_credentials = Credentials.from_service_account_file(
    'C://Users//Carmad94//Documents//workspace//data-engineering-zoom//level-agent-375808-5308078b1fc8.json'
)
bq_client = bigquery.Client(
    credentials=bq_credentials,
    project=project_id
)
storage_client = storage.Client(
    credentials=bq_credentials,
    project=project_id
)

In [54]:
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob("test.json")
blob.upload_from_filename(
    filename="test.json"
)

In [31]:
bigquery.SourceFormat.NEWLINE_DELIMITED_JSON

'NEWLINE_DELIMITED_JSON'

In [55]:
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    create_disposition=bigquery.CreateDisposition.CREATE_IF_NEEDED,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    schema_update_options=bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION,
    max_bad_records=200
)

uri = f"gs://{bucket_name}/test.json"
load_job = bq_client.load_table_from_uri(
    uri,
    "level-agent-375808.sample_weather.forecast",
    job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
print("Loaded {} rows.".format(load_job.output_rows))

Loaded 40 rows.


### Current weather

In [56]:
test_current = requests.get("https://api.openweathermap.org/data/2.5/weather?lat=7.0648306&lon=125.6080623&appid=bfc39b36158f668bf1412ec68685cd67&units=metric")

In [58]:
test_current.json().keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [64]:
current_df = pd.json_normalize(test_current.json(), max_level=2)
current_df.columns

Index(['weather', 'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod',
       'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'wind.speed', 'wind.deg', 'clouds.all', 'sys.type', 'sys.id',
       'sys.country', 'sys.sunrise', 'sys.sunset'],
      dtype='object')

In [71]:
current_df=pd.json_normalize(
    test_current.json(),
    'weather',
    [
        'base',
        'visibility',
        'dt',
        'timezone',
        'id',
        'name',
        'cod',
        ['coord','lon'],
        ['coord','lat'],
        ['main', 'temp'],
        ['main', 'feels_like'],
        ['main', 'temp_min'],
        ['main', 'temp_max'],
        ['main', 'pressure'],
        ['main', 'humidity'],
        ['wind', 'speed'],
        ['wind', 'deg'],
        ['wind', 'gust'],
        ['clouds', 'all'],
        ['sys', 'type'],
        ['sys', 'id'],
        ['sys', 'country'],
        ['sys', 'sunrise'],
        ['sys', 'sunset']
    ],
    errors='ignore',
    record_prefix='weather_'
)
current_df

,weather_id,weather_main,weather_description,weather_icon,base,visibility,dt,timezone,id,name,...,main.humidity,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,801,Clouds,few clouds,02d,stations,10000,1681804661,28800,1715348,Davao City,...,62,3.6,160,NaN,20,1,8154,PH,1681766888,1681811135


In [72]:
current_df.columns = current_df.columns.str.replace(".", "_")
current_df.columns

C:\Users\Carmad94\AppData\Local\Temp\ipykernel_8392\2779635513.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  current_df.columns = current_df.columns.str.replace(".", "_")


Index(['weather_id', 'weather_main', 'weather_description', 'weather_icon',
       'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod',
       'coord_lon', 'coord_lat', 'main_temp', 'main_feels_like',
       'main_temp_min', 'main_temp_max', 'main_pressure', 'main_humidity',
       'wind_speed', 'wind_deg', 'wind_gust', 'clouds_all', 'sys_type',
       'sys_id', 'sys_country', 'sys_sunrise', 'sys_sunset'],
      dtype='object')

In [73]:
filename='current.json'
current_df.to_json(
    filename,
    orient="records",
    lines=True
)

In [76]:
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob("current.json")
blob.upload_from_filename(
    filename="current.json"
)

In [77]:
uri = f"gs://{bucket_name}/current.json"
load_job = bq_client.load_table_from_uri(
    uri,
    "level-agent-375808.sample_weather.current",
    job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
print("Loaded {} rows.".format(load_job.output_rows))

Loaded 1 rows.


In [1]:
import datetime

In [1]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

now = 2023-04-19 14:58:25.115529
date and time = 19/04/2023 14:58:25


In [2]:
df = pd.read_csv('gs://sample-weather/cities in mindanao.csv')
df.head()
# pip install gcsfs

C:\Users\Carmad94\Envs\data-engineering-zoom\lib\site-packages\google\auth\_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,Name,Latitude,Longitude
0,Cagayan de Oro,8.475642,124.642153
1,Davao,7.064831,125.608062
2,Zamboanga,6.904691,122.076462
3,Butuan,8.948332,125.536944
4,General Santos,6.112331,125.171714


In [16]:
def test_sum(lat):
    return lat[0]+lat[0]

In [17]:
df[['Latitude', 'Longitude']].apply(test_sum, axis=1, result_type='expand')

0    16.951283
1    14.129661
2    13.809382
3    17.896665
4    12.224662
5    14.447526
6    13.902812
7    15.805769
dtype: float64

In [11]:
df[['Latitude', 'Longitude']]

,Latitude,Longitude
0,8.475642,124.642153
1,7.064831,125.608062
2,6.904691,122.076462
3,8.948332,125.536944
4,6.112331,125.171714
5,7.223763,124.246706
6,6.951406,126.216727
7,7.902884,125.089803


In [30]:
test_array = []
for row in df.itertuples():
    test_array.append(fetch(row.Latitude, row.Longitude))
len(test_array)

{'coord': {'lon': 124.6422, 'lat': 8.4756}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 303.38, 'feels_like': 308.25, 'temp_min': 303.38, 'temp_max': 303.38, 'pressure': 1007, 'humidity': 68, 'sea_level': 1007, 'grnd_level': 1006}, 'visibility': 10000, 'wind': {'speed': 2.12, 'deg': 340, 'gust': 2}, 'rain': {'1h': 0.15}, 'clouds': {'all': 100}, 'dt': 1682061238, 'sys': {'country': 'PH', 'sunrise': 1682026177, 'sunset': 1682070633}, 'timezone': 28800, 'id': 1721080, 'name': 'Cagayan de Oro', 'cod': 200}
{'coord': {'lon': 125.6081, 'lat': 7.0648}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 305.2, 'feels_like': 310.96, 'temp_min': 305.2, 'temp_max': 305.2, 'pressure': 1005, 'humidity': 62}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 240}, 'clouds': {'all': 20}, 'dt': 1682061202, 'sys': {'type': 1, 'id': 8154, 'country': '

8